In [6]:
## 2019-07-19
# Class 
# 1. 객체모델의 수단
# 2. ADT ( abstract data type ) - 추상적 데이터 타입

# Class안넹 정의 되는 내용 3가지
# 상태값 저장을 위한 변수 (field, member Variable, property)
# 수행하는 작업을 위한 함수 (method, member function, method)
# 정의된 class 정보를 바탕으로 일정 메모리 확보
# => 이런 확보된 메모리 공간 = 인스턴스, 객체
# 이런 객체를 생성하기 위해 생성자가 호출되어야함

# 학생 3명
# 각 학생이 가지는 정보는 국어, 영어, 수학 점수
# 평균, 총점 구하기

class Student:
    # Constructor 생성자
    def __init__(self,s_name,s_kor,s_eng,s_math):
        # property를 생성하려면 self를 이용
        self.student_name = s_name
        self.kor = s_kor
        self.eng = s_eng
        self.math = s_math
        
    # method
    def get_total(self):
        self.total = self.kor + self.eng + self.math
        return self.total
    
stu1 = Student("홍길동",10,20,30)
print(stu1.get_total())


60


In [1]:
## Ensemble을 이용한 MNIST
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

## Graph 초기화
tf.reset_default_graph()

# 모델을 여러개 만들어야함 => model class를 만든다.
## Class Define

## 1. Modeling
class CnnModel:
    # 생성자(Constructor)가 나와야함 
    # def = define
    def __init__(self,sess,name): # model이름을 지정함 / sess: 나중에 학습(run)하고 Accuracy구할때 사용하기 위해 지정
        # property를 생성하려면 self를 이용 , self를 붙이면 전역변수 안붙이면 지역변수
        self.sess = sess
        self.name = name
        self.build_net()
        
    # Model graph를 생성하는 method
    def build_net(self):
        # pass :  코드가 없음을 의미 , 자바에서는 {} 이용
        
        # 입력 받은 이름으로 변수 명을 설정한다.
        with tf.variable_scope(self.name):
            
        # input place holders
            self.X = tf.placeholder(shape=[None,784], dtype=tf.float32) 
            self.Y = tf.placeholder(shape=[None,10], dtype=tf.float32)
            self.drop_rate = tf.placeholder(dtype=tf.float32)

            # Convolution  
            # 28x28x1로 사이즈 변환 
            X_img = tf.reshape(self.X,[-1,28,28,1])

            # Convolution Layer1 
            L1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu) 
            # Pooling Layer1 
            L1 = tf.layers.max_pooling2d(inputs=L1, pool_size=[2,2], padding="SAME", strides=2)

            # Convolution Layer2
            L2 = tf.layers.conv2d(inputs=L1, filters=64, kernel_size=[3,3], padding="SAME", strides=1, activation=tf.nn.relu) 
            # Pooling Layer2 
            L2 = tf.layers.max_pooling2d(inputs=L2, pool_size=[2,2], padding="SAME", strides=2)
            # Neural Network => 2차원을 4차원으로
            L2 = tf.reshape(L2,[-1,7*7*64])

            # Weight & bias 
            W1 = tf.get_variable("weight1", shape=[7*7*64, 256], initializer=tf.contrib.layers.xavier_initializer()) 
            b1 = tf.Variable(tf.random_normal([256]), name="bias1")
            _layer1 = tf.nn.relu(tf.matmul(L2,W1) +b1)
            layer1 = tf.layers.dropout(_layer1, rate=self.drop_rate)

            W2 = tf.get_variable("weight2", shape=[256, 10], initializer=tf.contrib.layers.xavier_initializer()) 
            b2 = tf.Variable(tf.random_normal([10]), name="bias2")
#             W2 =  tf.get_variable("weight2", shape=[256,256], initializer=tf.contrib.layers.xavier_initializer())
#             b2 = tf.Variable(tf.random_normal([256]), name="bias2")
#             _layer2 = tf.nn.relu(tf.matmul(layer1,W2) + b2)
#             layer2 = tf.nn.dropout(_layer2, keep_prob = keep_prob)

#             W3 = tf.get_variable("weight3", shape=[7*7*64,256], initializer=tf.contrib.layers.xavier_initializer()) 
#             b3 = tf.Variable(tf.random_normal([]), name="bias3")

            # Hypothesis
            logits = tf.matmul(layer1,W2) + b2  
            self.H = tf.nn.relu(logits)

            # Cost Function
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits, labels = self.Y))

            # train
            self.train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.cost)
            
    ## Model 학습
    def train_net(self,train_x_data, train_y_data, drop_rate): # 학습을 하려면 데이터가 필요 / rate : drop_out에 대한 비율
        # pass
        training_epoch = 10
        batch_size = 30 
        
        for step in range(training_epoch): 
            num_of_iter = int( train_x_data.shape[0] /  batch_size) 
            cost_val = 0
            # (x데이터,y데이터)
            start_idx = 0
            end_idx = 0
        for i in range(num_of_iter):
            # batch_x, batch_y = mnist.train.next_batch(batch_size) 
            start_idx = i * batch_size
            end_idx = start_idx + batch_size
            # if end_idx > :
            # end_idx = batch_size
        
            batch_x = train_x_data[start_idx:end_idx,:]
            batch_y = train_y_data[start_idx:end_idx,:]
        
            _, cost_val = self.sess.run([self.train,self.cost], feed_dict={self.X:batch_x , self.Y:batch_y, self.drop_rate:drop_rate})
        
        print("{0} 학습중, epoch: {1}, cost: {2}".format(self.name, training_epoch, cost_val))

    
    ## Model의 정확도 측정
    def get_accuracy(self,test_x_data, test_y_data, drop_rate): # 학습한 데이터에 대한 test data 필요
        # pass
        self.predict = tf.argmax(self.H,1)
        correct = tf.equal(self.predict, tf.argmax(self.Y,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

        result = self.sess.run(self.accuracy, feed_dict={self.X:test_x_data, self.Y:test_y_data, self.drop_rate:drop_rate})

        print("{0}의 Accuracy: {1}".format(self.name, result))
    
    ## Model의 Prediction
    def get_prediction(self, x_data, drop_out_rate):
        # pass
        result = self.sess.run(self.predict, feed_dict={self.X: x_data, self.drop_rate: drop_out_rate})

    
#######################  여기까지는 그래프를 그리고 학습을 하고 정확도 측정, prediction
    
## 2. Data Loading
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

# Session 생성
sess = tf.Session()

## 3. Model의 개수를 지정하고 생성 = Ensemble
num_of_model = 10
cnn_models = [CnnModel(sess,"Model" + str(x)) for x in range(num_of_model)] # 10번돌면서 [모델(세션, 모델의 이름) for 루프] 

for model in cnn_models:
    sess.run(tf.global_variables_initializer())
    model.train_net(mnist.train.images, mnist.train.labels,0.3)
    model.get_accuracy(mnist.test, 0.3)
    model.get_prediction(mnist.test.images, 0.3)
    
## 4. 각각의 Model 학습

## 5. 각 Model의 accuracy 출력

## 6. ensemble의 accuracy 출력

## 7. Prediction

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./data/mnist\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./data/mnist\t10k-images-idx3-ubyte.gz
Extracting ./data/mnist\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dropout instead.
Model0 학습중, epoch: 10, cost: 0.12

TypeError: get_accuracy() missing 1 required positional argument: 'drop_rate'

In [ ]:
### 앙상블(Ensemble) => 모델을 여러개 만들어서 코드처리해야함. / 정확도는 증가하지만 속도는 느려짐(모델이 여러개 이여서)
### 앙상블은 이런 model이 여러개 있다.
### H1 => [0.5,,0.8, 0.99, 0.12, 0.34, ...]
### H2 => [0.2,,0.3, 0.94, 0.5, 0.1, ...]
### H3 => [0.7,,0.1, 0.3, 0.2, 0.12, ...]
### H4 => [0.26,,0.23, 0.194, 0.56, 0.31, ...]

### SUM => [1.66, 1.43, 2.4, 1.3, 1.2, ...] 컬럼별로 값들을 더함
### 최종 prediction은 SUM한 결과값을 가지고 예측 

In [ ]:
## Ensamble 을 이용한 MNIST

import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


##Graph 초기화
tf.reset_default_graph()



##Class Define
class CnnModel:
    #Constructor
    def __init__(self,sess,name):
        self.sess = sess
        self.name = name
        self.build_net()
      
    #Model graph 생성하는 method
    def build_net(self):
       # 입력 받은 이름으로 변수 명을 설정한다.
        with tf.variable_scope(self.name):

            # Boolean Tensor 생성 for dropout
            # tf.layers.dropout( training= True/Fals) True/False에 따라서 학습인지 / 예측인지 선택하게 됨
            # default = False
            self.training = tf.placeholder(tf.bool)

            # 입력 그래프 생성
            self.X = tf.placeholder(tf.float32, [None, 784])
            # 28x28x1로 사이즈 변환
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])

            # Convolutional Layer1
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3], padding='SAME', 
                                     activation=tf.nn.relu)
            # Pooling Layer1
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2,
                                            padding="SAME" )
            # Dropout Layer1
            dropout1 = tf.layers.dropout(inputs=pool1, rate=0.7, training=self.training)


            # Convolutional Layer2
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3], padding='SAME', activation=tf.nn.relu)
            # Pooling Layer2
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2],strides=2, padding='SAME' )
            # Dropout Layer2
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7, training=self.training)


            # Convolutional Layer3
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu)
            # Pooling Layer3
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2], strides=2, padding='SAME')
            # Dropout Layer3
            dropout3 = tf.layers.dropout(inputs=pool3, rate=0.7, training=self.training)

            # Dense Layer4 with Relu
            flat = tf.reshape(dropout3, [-1, 128*4*4])
            dense4 = tf.layers.dense(inputs=flat, units=625, activation=tf.nn.relu)
            # Dropout layer4
            dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=self.training)

            # Dense Layer5 with Relu
            dense5 = tf.layers.dense(inputs=dropout4, units=1050, activation=tf.nn.relu)
            # Dropout Layer5
            dropout5 = tf.layers.dropout(inputs=dense5, rate=0.5, training=self.training)


            # Logits layer : Final FC Layer5 Shape = (?, 625) -> 10
            self.logits = tf.layers.dense(inputs=dropout5, units=10)

        # Cost Function
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        # Test Model
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    #Model 학습
    def train_net(self,train_x_data,train_y_data,training=False):
        return self.sess.run([self.cost, self.optimizer], 
                             feed_dict={self.X:train_x_data, 
                                        self.Y:train_y_data,
                                        self.training:training})
     
   #Model의 Accuracy측정
    def get_accuracy(self,test_x_data,test_y_data,training=False):
        return self.sess.run(self.accuracy, 
                             feed_dict={self.X: test_x_data, 
                                        self.Y : test_y_data, 
                                        self.training:training})
    
   #Model의 Prediction
    def get_prediction(self,x_data,training=False):
        return self.sess.run(self.logits, feed_dict={self.X : x_data, self.training:training})
    


sess=tf.Session()
##1.Data Loading
mnist = input_data.read_data_sets("./data/mnist",
                                  one_hot=True)

# Hyper Prarameters
training_epochs = 15
batch_size = 100
learning_rate = 0.001

##2. Model 개수를 지정하고 생성
models = []
num_of_model = 2
#cnn_models = [CnnModel(sess,"Model" + str(x)) for x in range(num_of_model)]
for m in range(num_of_model):
    models.append(CnnModel(sess, "model"+str(m)))
                                                
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost_list = np.zeros(len(cnn_models))
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        # Train each model
        for m_idx, m in enumerate(cnn_models):
            c, _ = m.train_net(batch_xs, batch_ys)
            avg_cost_list[m_idx] += c / total_batch

    print('Epoch: ', '%04d' %(epoch + 1), 'Cost = ', avg_cost_list)
print('Training Finished')

                                                